# CEDA DataPoint - Demonstration

The CEDA DataPoint module wraps both the ``pystac_client`` and ``xarray`` packages into a single container for searching and accessing data. This notebook shows how methods function in DataPoint.

In [ ]:
from datapoint import DataPoint

client = DataPoint() # All public connection kwargs are known by default for CEDA

In [ ]:
# Same as normal for pystac_client

for coll in client.get_collections():
    print(f"{coll.id}: {coll.description}")

# Return a new client with a specified collection to search for - or results from pystac_client? (Could load lazily?)
sentinel2_ard = client.get_collection('sentinel2_ard')

# Make this a function or similar.
sentinel2_ard.extent.spatial.bboxes

In [ ]:
# Returns a new datapoint.SearchResult object. Inheritance for DataPoint, SearchResult from a general object type.
item_search = client.search(
    collections=['sentinel2_ard'],
    datetime="2023-01-01/2023-02-28",
    max_items=10,
)

In [ ]:
items = list(item_search.items())
items

In [ ]:
# Make all the properties into actual class properties?
for item in items:
    print(item.properties["start_datetime"], item.properties["end_datetime"])

In [ ]:
# Useful method for matching to catalog
item_search.url_with_parameters()
item_search.get_dois() # List item DOIs

In [ ]:
# Assemble xarray and other datasets
item_search.get_datasets()

# Could have datasets from multiple sources in the same Xarray object?